# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
# your code here
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Assuming your data is in 'tic-tac-toe.csv'
data = pd.read_csv('/content/tic-tac-toe.csv')

In [3]:
data.head()


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [4]:
# Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


In [5]:
# Convert the categorical values to numeric in all columns.

encoder = LabelEncoder()

for i in range(9):
    data.iloc[:, i] = encoder.fit_transform(data.iloc[:, i])

# Encode the 'class' column
data['class'] = encoder.fit_transform(data['class'])

In [6]:
data.head()


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,2,2,2,2,1,1,2,1,1,1
1,2,2,2,2,1,1,1,2,1,1
2,2,2,2,2,1,1,1,1,2,1
3,2,2,2,2,1,1,1,0,0,1
4,2,2,2,2,1,1,0,1,0,1


In [8]:
# Separate the inputs and output.

X = data.iloc[:, :-1]  # Features (grid positions)
y = data.iloc[:, -1]   # Target (class)


In [9]:
# Normalize the input data.

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X = scaler.fit_transform(X)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [14]:
# your code here
# 1. Split the training and test data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

In [130]:
import tensorflow as tf

tf.keras.utils.set_random_seed(55)

# 2. Create a Sequential model.
model = tf.keras.models.Sequential()

# 3. Add layers (ReLU for middle, Softmax for output).
model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(9,)))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

# 4.Compile the model (adam optimizer, sparse_categorical_crossentropy loss).
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# adam optimizer with custom learning rate

optimizer = tf.keras.optimizers.Adam(learning_rate=custom_learning_rate)
learning_rate=0.001
# 5-Fit the training data.
model.fit(X_train, y_train, epochs=66, batch_size=15)

# 6.Evaluate your neural network model with the test data.
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")
# 7. Save the model.
model.save('tic-tac-toe.keras')

Epoch 1/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5860 - loss: 0.6667
Epoch 2/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6390 - loss: 0.6218
Epoch 3/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6664 - loss: 0.5889
Epoch 4/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7346 - loss: 0.5557
Epoch 5/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7666 - loss: 0.5210
Epoch 6/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7950 - loss: 0.4849
Epoch 7/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8242 - loss: 0.4464
Epoch 8/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8482 - loss: 0.4088
Epoch 9/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8592 - loss: 0.3735
Epoch 10/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8736 - loss: 0.3404
Epoch 11/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8883 - loss: 0.3089
Epoch 12/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8921 - lo

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [142]:
# Make predictions using the model
predictions_proba = model.predict(X_test)
predictions = (predictions_proba[:,1] > 0.8)
from sklearn.metrics import accuracy_score
# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
accuracy

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


0.90625

In [ ]:
predictions_proba


In [144]:

predictions


array([False,  True, False, False, False,  True,  True, False,  True,
       False,  True,  True,  True, False,  True,  True,  True,  True,
       False, False, False,  True, False,  True, False,  True,  True,
        True, False,  True,  True,  True,  True,  True, False, False,
        True,  True,  True, False,  True,  True, False, False, False,
       False,  True,  True, False, False, False, False,  True,  True,
        True,  True,  True, False, False,  True,  True,  True, False,
        True,  True,  True,  True, False,  True,  True,  True, False,
       False,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True, False, False,  True,  True,
        True, False,  True, False,  True,  True, False,  True,  True,
        True,  True,  True, False,  True, False, False,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True, False,
        True, False, False,  True, False,  True, False, False,  True,
       False,  True,

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [131]:
# your code here
import tensorflow as tf

tf.keras.utils.set_random_seed(55)

# 2. Create a Sequential model.
model = tf.keras.models.Sequential()

# 3. Add layers (ReLU for middle, Softmax for output).
model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(9,)))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

# 4.Compile the model (adam optimizer, sparse_categorical_crossentropy loss).
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# adam optimizer with custom learning rate

optimizer = tf.keras.optimizers.Adam(learning_rate=custom_learning_rate)
learning_rate=0.001
# 5-Fit the training data.
model.fit(X_train, y_train, epochs=66, batch_size=15)

# 6.Evaluate your neural network model with the test data.
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")
# 7. Save the model.
model.save('tic-tac-toe.keras')

Epoch 1/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5860 - loss: 0.6667
Epoch 2/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6390 - loss: 0.6218
Epoch 3/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6664 - loss: 0.5889
Epoch 4/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7346 - loss: 0.5557
Epoch 5/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7666 - loss: 0.5210
Epoch 6/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7950 - loss: 0.4849
Epoch 7/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8242 - loss: 0.4464
Epoch 8/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8482 - loss: 0.4088
Epoch 9/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8592 - loss: 0.3735
Epoch 10/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8736 - loss: 0.3404
Epoch 11/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8883 - loss: 0.3089
Epoch 12/66
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8921 - lo

**Which approach(es) did you find helpful to improve your model performance?**

In [147]:
# Adding Layers
# Adjusting Learning Rate
# Adjusting Epochs